In [78]:
import pandas as pd
from scipy import spatial
import numpy as np
import geopy.distance
import timeit

In [79]:
#Read in port locations and distances
df_distances = pd.read_csv('Data/path/distances.csv')
df_ports = pd.read_csv('Data/path/ports.csv')
df_routes = pd.read_csv('Data/path/routes.csv')

#Read in level 1 solar plant locations [lon, lat, kWh/kWp yearly, average W/m2 yearly]
df_ren = pd.read_csv('Data/level1renewable.csv')

In [80]:
df_ports.sample(10)

,Unnamed: 0,PORT_NAME,INDEX_NO,coords
2878,15460,No3,NaN,"((39.32683585418208, -6.753334924578667),)"
1755,4988,CHIWAN,57818.0,"((110.02077865377504, 19.961165),)"
2636,28113,No 3,NaN,"((-71.38733091509586, -17.622709510500002),)"
1258,3461,bz 25-1 offshore,NaN,"((119.163335, 38.246665),)"
2493,26764,No 2,NaN,"((11.03205827394947, 55.248928014),)"
1789,5430,Cai Lan,NaN,"((107.05182309457581, 20.975740000000002),)"
2523,10582,NaN,NaN,"((177.41774069610983, -17.602322967723012),)"
1985,6792,NaN,NaN,"((-40.24325133343859, -22.667183333333334),)"
1867,6003,Ogdensburg,NaN,"((-75.48588802959502, 44.708215),)"
1261,3467,cabedelo,NaN,"((-34.840600581457664, -6.97153),)"


In [81]:
lons = df_ren['Longitude']
lats = df_ren['Latitude']

In [82]:
coords = df_ports['coords'].values.tolist()
coords = [i.strip('()') for i in coords]
coords = [i.strip("'),'") for i in coords]
coords = [i.split(', ') for i in coords]

coords2=[]
for i in range(len(coords)):
    li=[]
    for j in range(2):
                li.append(float(coords[i][j]))
    coords2.append(li)    

In [83]:
pt = [6, 30]  # <-- the point to find

nearest = coords2[spatial.KDTree(coords2).query(pt)[1]] # <-- the nearest point 

distance,index = spatial.KDTree(coords2).query(pt)

print(distance)
print(index)
coords2[index]


5.671611487518396
1815


[10.107777625000002, 33.910669999999996]

In [84]:
gen_coords = zip(lons,lats)
#closest = []
longs = []
latis = []
distance = []
port_code = []

for i in gen_coords:
    nearest = coords2[spatial.KDTree(coords2).query(i)[1]] # <-- the nearest point 
    distanceKD,index = spatial.KDTree(coords2).query(i)
    
    #closest.append(coords2[index])
    longs.append(coords2[index][0])
    latis.append(coords2[index][1])
    port_code.append(df_ports.loc[index,'Unnamed: 0'])
    distance.append(geopy.distance.distance((reversed(i)),(reversed(coords2[index]))))
    
df_ren['Port Code'] = port_code
df_ren['Port Long.'] = longs
df_ren['Port Lat.'] = latis
df_ren['Gen-Port Direct Dist.'] = distance

In [86]:
driving_distance = []

In [87]:
#Don't re-run unless necessary - getting driving distance from all gen. points to closest port. Takes about 50 mins

for i in range(0,len(df_ren['Longitude'])):
    try:
        # call the OSMR API
        r = requests.get(f"http://router.project-osrm.org/route/v1/car/{df_ren.loc[i,'Longitude']},{df_ren.loc[i,'Latitude']};{df_ren.loc[i,'Port Long.']},{df_ren.loc[i,'Port Lat.']}?overview=false""")
        # then you load the response using the json libray
        # by default you get only one alternative so you access 0-th element of the `routes`
        routes = json.loads(r.content)
        route_1 = routes.get("routes")[0]
        driving_distance.append(route_1["distance"])
    except:
        driving_distance.append(None)

df_ren['Gen-Port Driving Dist.'] = driving_distance

In [88]:
df_ren.to_csv('Data/renewables.csv')

In [1]:
df_ren.sample(10)

NameError: name 'df_ren' is not defined